# TinySquad Classes

In [4]:
import random

from enum import Enum, unique
from collections import namedtuple
from typing import List, NewType


CardType = Enum('CardType',[
    'HERO',
    'WEAPON',
    'HERO_EVENT',
    'HERO_UPGRADE',
    'HERO_POWER',
    'CHECK_MISS',
    'CHECK_HIT',
    'HERO_SUPPORT',
])


HeroActions = Enum('HeroActions',[
    'ATTACK',
    'MOVE',
    'ENGAGE',
    'DISENGAGE',
    'INTERACT',
    'CLIMB',
    'REST',
    'SECOND_CHANCE',
])


GameState = Enum('GameState',[
    'DRAFTING',
    'DEPLOYMENT',
    'START_OF_ROUND',
    'DETERMINE_INITIATIVE',
    'DRAW_CARDS',
    'PLAY_ROUND',
    'POINTS_CHECK',
    'END_OF_ROUND',
])


HeroState = Enum('HeroState',[
    'ACTIVE',
    'STUNNED',
    'FROZEN',
    'DISABLED'
])
    

Position = namedtuple('Position', ['x', 'y'])
ActionCost = namedtuple('ActionCost', ['cards','action_points','action_range'])


class PlayerBase:
    def __init__(self, name:str) -> None:
        self._name = name

    @property
    def name(self):
        return self._name



class Card:
    def __init__(self, card_type:CardType, name:str, owner:PlayerBase = None):
        self._type = card_type
        self._name = name
        self._owner = owner

    @property
    def name(self) -> str:
        return self._name

    @property
    def card_type(self) -> CardType:
        return self._type

    @property
    def owner(self) -> PlayerBase:
        return self._owner

    @owner.setter
    def owner(self, value:str):
        self._owner = value


class Deck:
    def __init__(self, cards:List[Card] = []):
        self._cards:List[Card] = cards if cards else []

    def queue_cards(self, cards: List[Card]):
        if cards:
            self._cards += cards

    def push_cards(self, cards: List[Card]):
        if cards:
            self._cards = cards + self._cards

    def merge(self, other):
        """
        """
        self._cards += other._cards

    def pop_cards(self, count:int=1) -> List[Card]:
        count = min(max(0, count), len(self._cards))
        cards = self._cards[0:count]
        if count:
            self._cards = self._cards[count:]
        return cards

    def shuffle(self):
        random.shuffle(self._cards)

    @property
    def count(self):
        return len(self._cards)

    def __iter__(self):
        return self._cards.__iter__()


class WeaponBase:
    def __init__(self, name:str, area:int, damage:int):
        self._name = name
        self._area = area
        self._damage = damage

    def can_attack(self, source, target) -> bool:
        return False

    def attack(self, source, target) -> bool:
        """
        Attacks a target, if all requirements are met
        """
        if self.can_attack(source, target):
            self._perform_attack(source, target)
            return True
        return False

    def _perform_attack(self, source, target):
        """
        Performs the attack if all requirements are met
        """
        pass

    def on_disabled(self):
        """
        Perform clean-up actions specific to the weapon
        """
        pass

    
class HeroBase:
    def __init__(self, name:str, life:int, desc:str, weapons=List[WeaponBase]):
        self._name = name
        self._life = life
        self._desc = desc
        self._state = HeroState.ACTIVE
        self._action_points = 0
        self._weapons = weapons
        self.position = Position(x=-1, y=-1)
        
    @property
    def name(self) -> str:
        return self._name
    
    @property
    def description(self) -> str:
        return self._desc
    
    @property
    def life(self) -> int:
        return self._life

    @property
    def is_active(self) -> bool:
        return self._state != HeroState.DISABLED

    @property
    def state(self) -> HeroState:
        return self._state

    @property
    def action_points(self) -> int:
        return self._action_points

    def add_action_points(self, points:int):
        '''
        Adds the AP to this Hero iff the points is greater than zero
        '''
        self._action_points += max(0, points)

    def consume_action_points(self, points:int) -> bool:
        '''
        Consumes action points, and returns "True" if it's viable, "False" otherwise.

        If consuming the requested AP isn't viable, no points will be consumed.
        '''
        remaining_points = self._action_points - max(0, points)
        if remaining_points >= 0:
            self._action_points = remaining_points
            return True
        return False

    @property
    def weapons(self) -> List[WeaponBase]:
        return self._weapons

    @property
    def position(self) -> Position:
        return self._position

    @position.setter
    def position(self, value:Position):
        self._position = value
        if self._position.x < 0 or self._position < 0:
            self._state = HeroState.DISABLED
            self._on_disabled()
    
    def consume_life(self, points:int) -> HeroState:
        self._life = max(0, self._life - points)
        if self._life <= 0:
            self._state = HeroState.DISABLED
            self._on_disabled()
        return self._state

    def _disable(self):
        if not self.is_active:
            return
        self._state = HeroState.DISABLED
        self._action_points = 0
        self._on_disabled()
        if not self._weapons:
            return
        for weapon in self._weapons:
            weapon.on_disabled()

    def _on_disabled(self):
        """
        Perform clean-up actions specific to the hero
        """
        pass


class Skill:
    def __init__(self, name:str):
        self._name = name
    
    @property
    def name(self):
        return self._name

    def on_game_state_changed(self, old_state:GameState, new_state:GameState):
        pass
    
    def consume_requirements(self, hero:HeroBase):
        pass


class Hero(HeroBase):
    def __init__(self, name:str, life:int, desc:str="", weapons=List[WeaponBase]):
        super().__init__(name=name, life=life, desc=desc)

    def __repr__(self) -> str:
        return f"{self.name} (Life: {self.life})"


class Player(PlayerBase):
    def __init__(self, name: str) -> None:
        super().__init__(name)


# TinySquad Data

In [5]:
import yaml

from enum import Enum, unique
from collections import namedtuple
from typing import Callable, Optional, List, Dict, Any

TargetType = Enum('TargetType', ['SELF', 'ALLY', 'OPPONENT'])

Targets = namedtuple('Targets', ['type', 'count', 'conditions'])



def is_valid_point(pos:Position) -> bool:
    return pos.x >= 0 and pos.y >= 0

def distance(pos1:Position, pos2:Position) -> Optional[int]:
    if not (is_valid_point(pos1) and is_valid_point(pos2)):
        # Distance calculation doesn't make sense if the point is outside the board
        return None
    if pos1.x == pos2.x:
        return abs(pos1.x - pos2.x)
    elif pos1.y == pos2.y:
        return abs(pos2.x - pos2.y)
    # The points are not in direct path
    return None

class Entity:
    def __init__(self) -> None:
        self._position:Position = None

    @property
    def position(self) -> Position:
        return self._position

    @position.setter
    def position(self, value:Position):
        self._position = value


class Cell(Entity):
    def __init__(self, position:Position) -> None:
        super().__init__()
        self._position = position
        self._entity = None

    @property
    def is_empty(self) -> bool:
        return self._entity is None

    @property
    def entity(self) -> Optional[Entity]:
        return self._entity

    @entity.setter
    def entity(self, value:Optional[Entity]):
        if isinstance(value, Cell):
            raise Exception(f"Cannot set the entity occupying a cell as another cell")
        self._entity = value

    @property
    def position(self) -> Position:
        return self._position


class Board(Entity):
    def __init__(self, rows:int, columns:int) -> None:
        super().__init__()
        self._rows = []
        self._row_size = rows
        self._columns_size = columns
        for y in range(rows):
            row = []
            for x in range(columns):
                row.append(Cell(Position(x=x, y=y)))
            self._rows.append(row)

    def is_valid_location(self, x:int, y:int) -> bool:
        return x >= 0 and x < self._columns_size and y >= 0 and y < self._row_size

    def is_valid_position(self, position: Position) -> bool:
        return self.is_valid_location(position.x, position.y)

    def get(self, x:int, y:int) -> Optional[Cell]:
        if not self.is_valid_location(x, y):
            return None
        return self._rows[y][x]


class RangeCheck:
    def __init__(self, impl:Callable[[int],bool]):
        self._impl = impl

    def is_in_range(self, pos1:Position, pos2:Position) -> bool:
        distance_between_points = distance(pos1, pos2)
        if distance_between_points is None:
            return False
        return self._impl(distance_between_points)


class Condition:
    def __init__(self, id:str):
        self._id = id

    def is_met(self, source:Entity, target:Entity, target_cell:Cell, board:Board) -> bool:
        return False


class TargetIsInRange(Condition):
    def __init__(self, range_check:Callable[[Position,Position],bool]):
        super().__init__('target_is_in_range')
        self._range_check = range_check

    def is_met(self, source:Entity, target:Entity, target_cell:Cell, board:Board) -> bool:
        return self._range_check(source.position, target.position)


class ConditionMissingFields(Exception):
    def __init__(self, condition_id:str, fields:List[str]):
        super().__init__(f"The condition '{condition_id}' is missing one or more fields; {fields}")


class InvalidValue(Exception):
    pass


class ConditionFactories:
    FACTORIES = {}

    @classmethod
    def add(cls, factory: 'ConditionFactory') -> 'ConditionFactory':
        cls.FACTORIES[factory.id] = factory
        return cls

    @classmethod
    def get(cls, condition_id: str) -> 'ConditionFactory':
        factory = cls.FACTORIES.get(condition_id)
        if factory is None:
            raise Exception(f"Invalid condition ID '{condition_id}'! Expected one of {cls.FACTORIES.keys()}.")
        return factory

    @classmethod
    def create(cls, data:Dict[Any, Any]) -> Condition:
        condition_id = data.get('id')
        if not condition_id:
            raise Exception(f"The condition data is missing 'id' to identify the type of the condition: {data}")
        return cls.get(condition_id=condition_id).create(data)


class ConditionFactory:
    def __init__(self, id:str, required_fields:List[str]):
        self._id = id
        self._required_fields = set(required_fields)

    @property
    def id(self) -> str:
        return self._id

    def create(self, data:Dict[Any, Any]) -> Condition:
        missing_fields = self._required_fields.difference(data.keys())
        if missing_fields:
            raise ConditionMissingFields(self.id, missing_fields)
        return self._create(data)

    def _create(self, data:Dict[Any, Any]) -> Condition:
        raise NotImplemented()


class TargetIsInRangeFactory(ConditionFactory):
    """
    Creates TargetIsInRange condition from dict
    """
    
    RANGE_CHECKERS = {
        'eq': lambda distance: RangeCheck(lambda value: value == distance),
        'lt': lambda distance: RangeCheck(lambda value: value < distance),
        'lte': lambda distance: RangeCheck(lambda value: value <= distance),
        'gt': lambda distance: RangeCheck(lambda value: value > distance),
        'gte': lambda distance: RangeCheck(lambda value: value >= distance),
    }

    def __init__(self):
        super().__init__('target_is_in_range', required_fields=['range'])
        self._range_fields = set(['op', 'value'])

    def _get_range_data(self, data:Dict[Any, Any]) -> Dict[Any, Any]:
        range_data = data['range']
        missing_range_fields = self._range_fields.difference(range_data.keys())
        if missing_range_fields:
            raise ConditionMissingFields(self.id, missing_range_fields)
        return range_data

    def _create(self, data:Dict[Any, Any]) -> Condition:
        range_data = self._get_range_data(data)

        range_op = range_data['op']
        if range_op not in TargetIsInRangeFactory.RANGE_CHECKERS:
            raise InvalidValue(f"{self.id}'s 'op' field has an invalid value. Found: {range_op}. Expected one of {TargetIsInRangeFactory.RANGE_CHECKERS.keys()}")

        return TargetIsInRangeFactory.RANGE_CHECKERS[range_op](range_data['value'])

ConditionFactories.add(TargetIsInRangeFactory())

TEST_YAML="""
conditions:
  - id: target_is_in_range
    range:
      op: eq
      value: 1
  - id: target_is_in_range
    range:
      op: gt
      value: 5
"""

data = yaml.safe_load(TEST_YAML)
conditions = []
for c in data['conditions']:
    conditions.append(ConditionFactories.create(c))

print(f"Created {len(conditions)} conditions!\n")

"""
Weapons
"""
class NaturesTouch(WeaponBase):
    def __init__(self):
        super().__init__(name="Nature's Touch", area=1, damage=1)

    def can_attack(self, source:Hero, target:Hero) -> bool:
        return False

"""
Hero Action Costs - Common to all heroes
"""
ACTION_COSTS = {
    HeroActions.ATTACK: ActionCost(cards=0, action_points=1, action_range=-1), # A -1 range means that the range is dependent
    HeroActions.MOVE: ActionCost(cards=0, action_points=1, action_range=3),
    HeroActions.INTERACT: ActionCost(cards=1, action_points=0, action_range=-1),
    HeroActions.CLIMB: ActionCost(cards=1, action_points=0, action_range=-1),
    HeroActions.SECOND_CHANCE: ActionCost(cards=1, action_points=0, action_range=-1),
    HeroActions.REST: ActionCost(cards=0, action_points=2, action_range=-1),
    HeroActions.ENGAGE: ActionCost(cards=2, action_points=0, action_range=1),
    HeroActions.DISENGAGE: ActionCost(cards=3, action_points=0, action_range=-1),
}

"""
Heroes
"""
HEROES = []
with open('heroes.yaml', 'r') as f:
    data = yaml.safe_load(f)
    for entry in data:
        HEROES.append(Hero(name=entry['name'], life=entry['life']))

for hero in HEROES:
    print(hero)

Created 2 conditions!

Ivy (Life: 7)
Sally (Life: 8)
Wonder (Life: 6)
Harry (Life: 6)
Lara (Life: 6)
Bat (Life: 7)
Daeny (Life: 6)
Quinn (Life: 6)
Super (Life: 6)
Ripley (Life: 6)
Alice (Life: 6)
Furiosa (Life: 7)


## Condition Tests

In [9]:
from typing import Any, Dict
import unittest


class TestConditions(unittest.TestCase):
    def _load_yaml_data(self, relative_filename: str) -> Dict[Any, Any]:
        with open(f"test_data/{relative_filename}.yaml", 'r') as f:
            return yaml.safe_load(f)

    def _load_condition(self, test_file_name:str):
        print(f"Loading condition in {test_file_name}... ", end='')
        self.condition = ConditionFactories.create(self._load_yaml_data(test_file_name))
        print("[DONE]")

    def test_target_is_in_range_eq_1(self):
        self._load_condition('target_is_in_range_eq_1')
        ent1 = Entity()
        ent1.position = Position(x=10, y=10)
        ent2 = Entity()
        ent2.position = Position(x=11, y=10)
        self.assertTrue(self.condition.is_in_range(ent1.position, ent2.position))


unittest.main(argv=[''], verbosity=2, exit=False)

test_target_is_in_range_eq_1 (__main__.TestConditions) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


Loading condition in target_is_in_range_eq_1... [DONE]
